In [ ]:
import pandas as pd
df_gasoline = pd.read_csv("../tests/gasoline.csv", index_col=0, parse_dates=True)
df_gasoline.head()

In [ ]:
# plot the data
import pandas as pd
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
df_gasoline['value'].plot()
plt.title('Gasoline Prices in the US - Weekly')
plt.xlabel('Date')
plt.ylabel('Dollars per Gallon')
plt.savefig('../results/gasoline.png')
plt.show()

## Seasonality in the data

We seek to examine the seasonality in the data. We will do this by looking at the autocorrelation function (ACF) and partial autocorrelation function (PACF) of the data. We will also look at the periodogram of the data.

In [ ]:
# ACF and PACF plots:

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plt.rc("figure", figsize=(20,10))
plot_acf(df_gasoline['value'], lags=20)
plt.savefig('../results/gasoline_acf.png')
plt.show()

plot_pacf(df_gasoline['value'], lags=20, method='ywm')
plt.savefig('../results/gasoline_pacf.png')
plt.show()

In [ ]:
from pandas.plotting import autocorrelation_plot

autocorrelation_plot(df_gasoline['value'])
plt.savefig('../results/gasoline_autocorrelation.png')
plt.show()

# Standard Linear regression

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import math


data = df_gasoline['value']
data = np.array(data)
# scale the data
data = (data - data.mean()) / (data.max() - data.min())

In [ ]:
test_size = 120

In [ ]:
x_train = np.arange(0, len(data) - test_size).reshape(-1, 1)
y_train = data[:len(x_train)]
x_test = np.arange(len(data) - test_size, len(data)).reshape(-1, 1)
y_test = data[len(x_train):]

In [ ]:
model_linear_reg = LinearRegression().fit(x_train, y_train)
y_predicted = model_linear_reg.predict(x_test)

# plot predictions
x_train_plot = np.arange(0, len(x_train))
x_test_plot = np.arange(len(x_train), len(x_train) + len(x_test))

plt.figure(figsize=(20, 10))
plt.plot(x_train_plot, y_train)
plt.plot(x_test_plot, y_test)
plt.plot(x_test_plot, y_predicted)
plt.legend(['train', 'test', 'predicted'])
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(x_test_plot, y_test)
plt.plot(x_test_plot, y_predicted)
plt.legend([ 'test', 'predicted'])
plt.ylim(-0.2, 0.6)
plt.savefig('../results/gasoline_linear_regression.png')
plt.show()

print("RMSE: ", math.sqrt(mean_squared_error(y_test, y_predicted)))

# ARIMA Model - for capturing seasonality

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from matplotlib import pyplot

data = df_gasoline['value'].values


# fit model
model = ARIMA(data, order=(5,1,0))
model_fit = model.fit()
# summary of fit model
print(model_fit.summary())
# line plot of residuals
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()
pyplot.show()
# density plot of residuals
residuals.plot(kind='kde')
pyplot.show()
# summary stats of residuals
print(residuals.describe())

In [ ]:
from pandas.plotting import autocorrelation_plot

plt.figure(figsize=(20, 10))
autocorrelation_plot(data)
plt.show()

## Using a rolling forecast ARIMA model

In [ ]:
# evaluate an ARIMA model using a walk-forward validation
from matplotlib import pyplot
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
# load dataset

series = df_gasoline['value']
# split into train and test sets
X = series.values
# normalize dataset
X = X.astype('float32')
X = (X - X.mean()) / (X.max() - X.min())

size = len(X) - test_size
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
# walk-forward validation
for t in tqdm(range(len(test))):
	model = ARIMA(history, order=(5,1,0))
	model_fit = model.fit()
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)

model_fit.plot_diagnostics(figsize=(20,10))
plt.show()

In [ ]:

# evaluate forecasts
rmse = math.sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)
# plot forecasts against actual outcomes
plt.plot(test)
plt.plot(predictions, color='red')
plt.ylim(-0.2, 0.6)
plt.legend(['test', 'predicted'])
plt.savefig('../results/gasoline_arima.png')
pyplot.show()

# Pyro

In [ ]:
import torch
import pyro
import pyro.distributions as dist
import pyro.poutine as poutine
from pyro.contrib.examples.bart import load_bart_od
from pyro.contrib.forecast import ForecastingModel, Forecaster, backtest, eval_crps
from pyro.infer.reparam import LocScaleReparam, StableReparam
from pyro.ops.tensor_utils import periodic_cumsum, periodic_repeat, periodic_features
from pyro.ops.stats import quantile

In [ ]:
class Model1(ForecastingModel):
    # We then implement the .model() method. Since this is a generative model, it shouldn't
    # look at data; however it is convenient to see the shape of data we're supposed to
    # generate, so this inputs a zeros_like(data) tensor instead of the actual data.
    def model(self, zero_data, covariates):
        data_dim = zero_data.size(-1)  # Should be 1 in this univariate tutorial.
        feature_dim = covariates.size(-1)

        # The first part of the model is a probabilistic program to create a prediction.
        # We use the zero_data as a template for the shape of the prediction.
        bias = pyro.sample("bias", dist.Normal(0, 10).expand([data_dim]).to_event(1))
        weight = pyro.sample("weight", dist.Normal(0, 0.1).expand([feature_dim]).to_event(1))
        prediction = bias + (weight * covariates).sum(-1, keepdim=True)
        # The prediction should have the same shape as zero_data (duration, obs_dim),
        # but may have additional sample dimensions on the left.
        assert prediction.shape[-2:] == zero_data.shape

        # The next part of the model creates a likelihood or noise distribution.
        # Again we'll be Bayesian and write this as a probabilistic program with
        # priors over parameters.
        noise_scale = pyro.sample("noise_scale", dist.LogNormal(-5, 5).expand([1]).to_event(1))
        noise_dist = dist.Normal(0, noise_scale)

        # The final step is to call the .predict() method.
        self.predict(noise_dist, prediction)

In [ ]:
data = torch.tensor(data, dtype=torch.float)
# scale data to be between -1 and 1
data = (data - data.mean()) / (data.max() - data.min())

# add observation dimension
data.unsqueeze_(-1)

In [ ]:
base = 0
end = data.size(-2)
mid = end - test_size
pyro.set_rng_seed(1)
pyro.clear_param_store()
time = torch.arange(float(end)) / 52


In [ ]:
covariates = torch.stack([time], dim=-1)
print(covariates.shape, data.shape)


In [ ]:
forecaster = Forecaster(Model1(), data[:mid], covariates[:mid], learning_rate=0.1)

In [ ]:
samples = forecaster(data[:mid], covariates, num_samples=1000)
p10, p50, p90 = quantile(samples, (0.1, 0.5, 0.9)).squeeze(-1)
crps = eval_crps(samples, data[mid:])
print(samples.shape, p10.shape)

plt.figure(figsize=(20, 10))
plt.fill_between(torch.arange(mid, end), p10, p90, color="red", alpha=0.3)
plt.plot(torch.arange(mid, end), p50, 'r-', label='forecast')
plt.plot(data, 'k-', label='truth')
plt.xlim(0, None)
plt.legend(loc="best")
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.fill_between(torch.arange(mid, end), p10, p90, color="red", alpha=0.3)
plt.plot(torch.arange(mid, end), p50, 'r-', label='forecast')
plt.plot(torch.arange(mid, end), data[mid:], 'k-', label='truth')
plt.xlim(mid, None)
plt.ylim(-0.2, 0.6)
plt.legend(loc="best")
plt.savefig('../results/gasoline_pyro_m1.png')
plt.show()

print("RMSE: ", math.sqrt(mean_squared_error(data[mid:], p50)))

In [ ]:
pyro.set_rng_seed(1)
pyro.clear_param_store()
time = torch.arange(float(end)) / 365
covariates = torch.cat([time.unsqueeze(-1),
                        periodic_features(end, 52)], dim=-1)
forecaster = Forecaster(Model1(), data[:mid], covariates[:mid], learning_rate=0.1)

In [ ]:
samples = forecaster(data[:mid], covariates, num_samples=1000)
p10, p50, p90 = quantile(samples, (0.1, 0.5, 0.9)).squeeze(-1)
crps = eval_crps(samples, data[mid:])

plt.figure(figsize=(20, 10))
plt.fill_between(torch.arange(mid, end), p10, p90, color="red", alpha=0.3)
plt.plot(torch.arange(mid, end), p50, 'r-', label='forecast')
plt.plot(data, 'k-', label='truth')
plt.xlim(0, None)
plt.legend(loc="best")
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.fill_between(torch.arange(mid, end), p10, p90, color="red", alpha=0.3)
plt.plot(torch.arange(mid, end), p50, 'r-', label='forecast')
plt.plot(torch.arange(mid, end), data[mid:], 'k-', label='truth')
plt.xlim(mid, None)
plt.ylim(-0.2, 0.6)
plt.legend(loc="best")
plt.savefig('../results/gasoline_pyro_m1_seasonal.png')
plt.show()

print("RMSE: ", math.sqrt(mean_squared_error(data[mid:], p50)))

In [ ]:
class Model2(ForecastingModel):
    def model(self, zero_data, covariates):
        data_dim = zero_data.size(-1)
        feature_dim = covariates.size(-1)
        bias = pyro.sample("bias", dist.Normal(0, 10).expand([data_dim]).to_event(1))
        weight = pyro.sample("weight", dist.Normal(0, 0.1).expand([feature_dim]).to_event(1))

        # We'll sample a time-global scale parameter outside the time plate,
        # then time-local iid noise inside the time plate.
        drift_scale = pyro.sample("drift_scale",
                                  dist.LogNormal(-20, 5).expand([1]).to_event(1))
        with self.time_plate:
            # We'll use a reparameterizer to improve variational fit. The model would still be
            # correct if you removed this context manager, but the fit appears to be worse.
            with poutine.reparam(config={"drift": LocScaleReparam()}):
                drift = pyro.sample("drift", dist.Normal(zero_data, drift_scale).to_event(1))

        # After we sample the iid "drift" noise we can combine it in any time-dependent way.
        # It is important to keep everything inside the plate independent and apply dependent
        # transforms outside the plate.
        motion = drift.cumsum(-2)  # A Brownian motion.

        # The prediction now includes three terms.
        prediction = motion + bias + (weight * covariates).sum(-1, keepdim=True)
        assert prediction.shape[-2:] == zero_data.shape

        # Construct the noise distribution and predict.
        noise_scale = pyro.sample("noise_scale", dist.LogNormal(-5, 5).expand([1]).to_event(1))
        noise_dist = dist.Normal(0, noise_scale)
        self.predict(noise_dist, prediction)

In [ ]:
pyro.set_rng_seed(1)
pyro.clear_param_store()
time = torch.arange(float(end)) / 365
# covariates = torch.stack([time], dim=-1)
covariates = torch.cat([time.unsqueeze(-1),
                        periodic_features(end, 52)], dim=-1)
forecaster = Forecaster(Model2(), data[:mid], covariates[:mid], learning_rate=0.1, time_reparam="dct", num_steps=1000)

In [ ]:
samples = forecaster(data[:mid], covariates, num_samples=1000)
p10, p50, p90 = quantile(samples, (0.1, 0.5, 0.9)).squeeze(-1)
crps = eval_crps(samples, data[mid:])

plt.figure(figsize=(20, 10))
plt.fill_between(torch.arange(mid, end), p10, p90, color="red", alpha=0.3)
plt.plot(torch.arange(mid, end), p50, 'r-', label='forecast')
plt.plot(data, 'k-', label='truth')
plt.xlim(0, None)
plt.legend(loc="best")
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.fill_between(torch.arange(mid, end), p10, p90, color="red", alpha=0.3)
plt.plot(torch.arange(mid, end), p50, 'r-', label='forecast')
plt.plot(torch.arange(mid, end), data[mid:], 'k-', label='truth')
plt.xlim(mid, None)
plt.ylim(-0.2, 0.6)
plt.legend(loc="best")
plt.savefig('../results/gasoline_pyro_m2.png')
plt.show()

print("RMSE: ", math.sqrt(mean_squared_error(data[mid:], p50)))

In [ ]:
class Model3(ForecastingModel):
    def model(self, zero_data, covariates):
        data_dim = zero_data.size(-1)
        feature_dim = covariates.size(-1)
        bias = pyro.sample("bias", dist.Normal(0, 10).expand([data_dim]).to_event(1))
        weight = pyro.sample("weight", dist.Normal(0, 0.1).expand([feature_dim]).to_event(1))

        drift_scale = pyro.sample("drift_scale", dist.LogNormal(-20, 5).expand([1]).to_event(1))
        with self.time_plate:
            with poutine.reparam(config={"drift": LocScaleReparam()}):
                drift = pyro.sample("drift", dist.Normal(zero_data, drift_scale).to_event(1))
        motion = drift.cumsum(-2)  # A Brownian motion.

        prediction = motion + bias + (weight * covariates).sum(-1, keepdim=True)
        assert prediction.shape[-2:] == zero_data.shape

        # The next part of the model creates a likelihood or noise distribution.
        # Again we'll be Bayesian and write this as a probabilistic program with
        # priors over parameters.
        stability = pyro.sample("noise_stability", dist.Uniform(1, 2).expand([1]).to_event(1))
        skew = pyro.sample("noise_skew", dist.Uniform(-1, 1).expand([1]).to_event(1))
        scale = pyro.sample("noise_scale", dist.LogNormal(-5, 5).expand([1]).to_event(1))
        noise_dist = dist.Stable(stability, skew, scale)

        # We need to use a reparameterizer to handle the Stable distribution.
        # Note "residual" is the name of Pyro's internal sample site in self.predict().
        with poutine.reparam(config={"residual": StableReparam()}):
            self.predict(noise_dist, prediction)

In [ ]:
pyro.set_rng_seed(2)
pyro.clear_param_store()
time = torch.arange(float(end)) / 365
covariates = periodic_features(end, 52)
forecaster = Forecaster(Model3(), data[:mid], covariates[:mid], learning_rate=0.1,
                        time_reparam="dct")
for name, value in forecaster.guide.median().items():
    if value.numel() == 1:
        print("{} = {:0.4g}".format(name, value.item()))

In [ ]:
samples = forecaster(data[:mid], covariates, num_samples=1000)
p10, p50, p90 = quantile(samples, (0.1, 0.5, 0.9)).squeeze(-1)
crps = eval_crps(samples, data[mid:])

plt.figure(figsize=(20, 10))
plt.fill_between(torch.arange(mid, end), p10, p90, color="red", alpha=0.3)
plt.plot(torch.arange(mid, end), p50, 'r-', label='forecast')
plt.plot(data, 'k-', label='truth')
plt.xlim(0, None)
plt.legend(loc="best")
plt.savefig('../results/gasoline_pyro_m3.png')
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.fill_between(torch.arange(mid, end), p10, p90, color="red", alpha=0.3)
plt.plot(torch.arange(mid, end), p50, 'r-', label='forecast')
plt.plot(torch.arange(mid, end), data[mid:], 'k-', label='truth')
plt.xlim(mid, None)
plt.legend(loc="best")
plt.savefig('../results/gasoline_pyro_m3.png')
plt.show()

print("RMSE: ", math.sqrt(mean_squared_error(data[mid:], p50)))

## Backtesting using pyro - evaluating model performance

In [ ]:
pyro.set_rng_seed(1)
pyro.clear_param_store()
windows1 = backtest(data, covariates, Model1,
                    min_train_window=62, test_window=100, stride=50,
                    forecaster_options={"learning_rate": 0.1, "time_reparam": "dct",
                                        "log_every": 1000, "warm_start": True})

In [ ]:
pyro.set_rng_seed(1)
pyro.clear_param_store()
windows2 = backtest(data, covariates, Model2,
                    min_train_window=62, test_window=100, stride=50,
                    forecaster_options={"learning_rate": 0.1, "time_reparam": "dct",
                                        "log_every": 1000, "warm_start": True})

In [ ]:
pyro.set_rng_seed(1)
pyro.clear_param_store()
windows3 = backtest(data, covariates, Model3,
                    min_train_window=62, test_window=100, stride=50,
                    forecaster_options={"learning_rate": 0.1, "time_reparam": "dct",
                                        "log_every": 1000, "warm_start": True})

In [ ]:
fig, axes = plt.subplots(3, figsize=(20, 20), sharex=True)

axes[0].plot([w["crps"] for w in windows2], "bx", label="local gaussian")
axes[0].plot([w["crps"] for w in windows1], "ro", label="global gaussian")
axes[0].plot([w["crps"] for w in windows3], "g+", label="heavy-tailed Stable")
# draw line joining the points
axes[0].plot([w["crps"] for w in windows2], "b", alpha=0.3)
axes[0].plot([w["crps"] for w in windows1], "r", alpha=0.3)
axes[0].plot([w["crps"] for w in windows3], "g", alpha=0.3)
axes[0].set_ylabel("CRPS")

axes[1].plot([w["mae"] for w in windows2], "bx", label="local gaussian")
axes[1].plot([w["mae"] for w in windows1], "ro", label="global gaussian")
axes[1].plot([w["mae"] for w in windows3], "g+", label="heavy-tailed Stable")
# draw line joining the points
axes[1].plot([w["mae"] for w in windows2], "b", alpha=0.3)
axes[1].plot([w["mae"] for w in windows1], "r", alpha=0.3)
axes[1].plot([w["mae"] for w in windows3], "g", alpha=0.3)
axes[1].set_ylabel("MAE")

axes[2].plot([w["rmse"] for w in windows2], "bx", label="local gaussian")
axes[2].plot([w["rmse"] for w in windows1], "ro", label="global gaussian")
axes[2].plot([w["rmse"] for w in windows3], "g+", label="heavy-tailed Stable")
# draw line joining the points
axes[2].plot([w["rmse"] for w in windows2], "b", alpha=0.3)
axes[2].plot([w["rmse"] for w in windows1], "r", alpha=0.3)
axes[2].plot([w["rmse"] for w in windows3], "g", alpha=0.3)
axes[2].set_ylabel("RMSE")

axes[0].legend(loc="best")
plt.savefig('../results/gasoline_pyro_comparison.png')
plt.tight_layout()